# Workshop on Statistical Genetics and Genetic Epidemiology STAGE-Quebec
## Theme 2 - Molecular Phenotypes in Genetic Epidemiology

By Marc-André Legault (Université de Montréal) and Qihuang Zhang (McGill University)

**July 31 - August 1, 2025**

### Introduction

In this workshop, we will conduct a transcriptome-wide association analysis (TWAS) to identify type 2 diabetes (T2D) genes. More precisely, we will test the association between predicted gene expression in various tissues and T2D. We will use real summary association statistics from a recent meta-analysis involving 180,834 cases with T2D and over a million controls [Mahajan _et al._ (2022)](https://www.nature.com/articles/s41588-022-01058-3). We will use both the [S-PrediXcan](https://www.nature.com/articles/s41467-018-03621-1) and [TWAS FUSION](https://www.nature.com/articles/ng.3506) to conduct the TWAS analyses. The GWAS summary statistics have been pre-processed to ensure they are on the same genome build (GRCh38) as the gene expression prediction models distributed by these software. We will also focus on a subset of tissues from GTEx to reduce the computational burden.

We split the workshop contents into three sections.

1. S-PrediXcan (this Notebook)
2. TWAS / FUSION [(link)](2a-FUSION.ipynb)
3. Comparing both approaches [(link)](3a-compare_TWAS_models.ipynb)

There is a file browser on the left pane that can be used to browse the workshop contents.

<div class="alert alert-warning">
    <p>
    The <strong>local</strong> folder is shared with your computer. Everything else will not be saved if you close the notebook server! Make sure that everything that should persist is store in the local folder.
    </p>
    <p>
    The contents of this folder are stored in your home directory (e.g. C:\Users\[your username]\STAGE2025_workshop_theme2 or <br /> /Users/[your username]/STAGE2025_workshop_theme2).
    </p>
</div>

We can now begin by better understanding S-PrediXcan and running it using our pre-processed GWAS data.

### Exploring the GWAS summary statistics

The GWAS summary statistics have been harmonized with [PredictDB](https://predictdb.hakyimlab.org/) models which are the gene expression predictions models distributed as companion data to S-PrediXcan. The data format is the same for every tissue:

In [1]:
filename="../data/PrediXcan/gwas_harmonized/harmonized_en_Whole_Blood.tsv.gz"
echo "Looking at file $filename"
echo "Column numbers and titles:"
echo "$(zcat $filename | head -n 1 | tr '\t' '\n' | nl)"
echo
echo "File excerpt:"
zcat $filename | head -n 10 | cut -f 1-6,9

Looking at file ../data/PrediXcan/gwas_harmonized/harmonized_en_Whole_Blood.tsv.gz
Column numbers and titles:
     1	chromosome
     2	base_pair_location
     3	effect_allele
     4	other_allele
     5	beta
     6	standard_error
     7	effect_allele_frequency
     8	neg_log_10_p_value
     9	rsid_x
    10	variant_id
    11	predictdb_id
    12	rsid_y

File excerpt:
chromosome	base_pair_location	effect_allele	other_allele	beta	standard_error	rsid_x
chr1	785910	C	G	-0.0176	0.0262	rs12565286
chr1	817186	A	G	0.0048	0.0098	rs3094315
chr1	817341	G	A	0.0045	0.0097	rs3131972
chr1	825532	T	C	0.0069	0.0099	rs1048488
chr1	825767	C	T	0.0066	0.0098	rs3115850
chr1	833068	A	G	0.0	0.0117	rs12562034
chr1	841742	T	A	0.0078	0.0101	rs2980319
chr1	843942	G	A	-0.007	0.0101	rs4040617
chr1	849670	A	G	0.0063	0.0098	rs2905062


<div class="alert alert-info">
    Interpret the displayed columns.
</div>

### Gene expression prediction models

Now, we can look at the gene expression prediction models for the GTEx tissue "Whole blood". In PredictDB, the models are distributed as sqlite3 databases. The databases have two tables: ``extra`` and ``weights`` with the following structure:

In [12]:
# Schema for the "extra" table:
sqlite3 ../data/PrediXcan/predictdb/en_Whole_Blood.db ".schema extra"

CREATE TABLE `extra` (
  `gene` TEXT,
  `genename` TEXT,
  `gene_type` TEXT,
  `alpha` REAL,
  `n_snps_in_window` INTEGER,
  `n.snps.in.model` INTEGER,
  `test_R2_avg` REAL,
  `test_R2_sd` REAL,
  `cv_R2_avg` REAL,
  `cv_R2_sd` REAL,
  `in_sample_R2` REAL,
  `nested_cv_fisher_pval` REAL,
  `nested_cv_converged` INTEGER,
  `rho_avg` REAL,
  `rho_se` REAL,
  `rho_zscore` REAL,
  `pred.perf.R2` REAL,
  `pred.perf.pval` REAL,
  `pred.perf.qval` REAL,
  `phi` REAL
);
CREATE INDEX metab_model_summary ON extra (gene);


``extra`` contains metadata on the predictive model for every gene. You will find the gene name, the Ensembl gene ID, the number of SNPs that were included in the model along with statistics on the predictive performance of the gene expression model. 

<div class="alert alert-info">
Comment on the importance of the <code>pred.perf.pval</code> column. What is it for?
</div>

The weights table contains the information on the gene expression model. For every ``gene`` (Ensembl gene ID), it will list the variants that are used to predict gene expression (i.e. eQTLs) along with their prediction ``weight``. As a concrete example, we can extract the model used to predict the expression for ENSG00000157823.16 (Adaptor Related Protein Complex 3 Subunit Sigma 2, _AP3S2_). For brevity, we will sort the variants by absolute weight, and only show the top 10 contributors to the prediction.

In [5]:
sqlite3 ../data/PrediXcan/predictdb/en_Whole_Blood.db <<EOF
.mode columns
.header on
select * from weights
  where gene='ENSG00000157823.16'
  order by abs(weight) desc limit 10;
EOF

gene                rsid        varID                   ref_allele  eff_allele  weight             
------------------  ----------  ----------------------  ----------  ----------  -------------------
ENSG00000157823.16  rs4363847   chr15_90547112_T_C_b38  T           C           0.128329668848438  
ENSG00000157823.16  rs12898828  chr15_89817721_A_T_b38  A           T           -0.109067828981916 
ENSG00000157823.16  rs11073814  chr15_88835653_A_T_b38  A           T           0.0524825659916266 
ENSG00000157823.16  rs3853641   chr15_89900547_G_C_b38  G           C           -0.0467714907373544
ENSG00000157823.16  rs16943673  chr15_89864201_T_C_b38  T           C           -0.0326710086433488
ENSG00000157823.16  rs10520684  chr15_89881770_C_A_b38  C           A           -0.0295110487238457
ENSG00000157823.16  rs1879530   chr15_88902553_G_A_b38  G           A           0.0263159897074107 
ENSG00000157823.16  rs2165069   chr15_89863887_C_T_b38  C           T           -0.0251527944185157


<div class="alert alert-info">
Why are some weights positive and some negative? Can you mention a way of estimating the prediction weights?
</div>

### TWAS analysis using S-PrediXcan

Now that we have explored the structure of PredictDB gene expression prediction models and that we have GWAS summary statistics, we can implement the TWAS analysis using S-PrediXcan. The code below may run for a few minutes. If it takes too long, you can stop it using the stop button at the top of this window, and use the checkpoint files available in the repository.

The call to SPrediXcan takes 3 input files:

- model_db_path: This contains the weights to predict the expression of the genes included in the TWAS. It is the sqlite3 database provided by PredictDB.
- covariance: This file contains LD estimates between the SNPs. It accounts for the covariance between genetic variants.
- gwas_file: This is the GWAS summary statistics files containing the association statistics between genetic variants and T2D. We previously harmonized the raw GWAS results to make sure that the variants were consistent with the ones available in PredictDB.

The parameters with the ``_column`` suffix are used so that the software knows how to interpret the GWAS summary statistics file. You can convince yourself that the names provided below are appropriate by looking back at the excerpt from the GWAS summary statistics file above.

Finally, we provide an estimate of the SNP heritability ($h^2$) for our phenotype and the GWAS sample size (N). These parameters allow the software to recalibrate the association statistics to account for polygenicity, as described in [Liang _et al. medRxiv_ 2024](https://www.biorxiv.org/content/10.1101/2023.10.17.562831v2). In short, polygenic traits have many causal variants with weak effect sizes scattered across the genome. Many of these variants will be in LD with the eQTLs included in the gene expression prediction models, which could lead to spurious associations. Providing these statistics allows S-PrediXcan to recalibrate the TWAS association statistics to account for this effect.

In [1]:
tissues="Adipose_Subcutaneous Artery_Coronary Brain_Cortex Liver Muscle_Skeletal Pancreas Whole_Blood"

for tissue in $tissues; do
    echo
    echo
    echo "Running S-PrediXcan for tissue ${tissue}"
    echo
    SPrediXcan.py \
        --model_db_path ../data/PrediXcan/predictdb/en_${tissue}.db \
        --covariance ../data/PrediXcan/predictdb/en_${tissue}.txt.gz \
        --gwas_file ../data/PrediXcan/gwas_harmonized/harmonized_en_${tissue}.tsv.gz \
        --snp_column rsid_x \
        --effect_allele_column effect_allele \
        --non_effect_allele_column other_allele \
        --output_file /workshop/local/results/twas_en_${tissue}.csv \
        --beta_column beta \
        --se_column standard_error \
        --gwas_h2 0.15 \
        --gwas_N 1000000
done



Running S-PrediXcan for tissue Adipose_Subcutaneous

INFO - Processing GWAS command line parameters
INFO - Building beta for ../data/PrediXcan/gwas_harmonized/harmonized_en_Adipose_Subcutaneous.tsv.gz and ../data/PrediXcan/predictdb/en_Adipose_Subcutaneous.db
INFO - Reading input gwas with special handling: ../data/PrediXcan/gwas_harmonized/harmonized_en_Adipose_Subcutaneous.tsv.gz
INFO - Processing input gwas
INFO - Calculating zscore from se and beta
INFO - Aligning GWAS to models
INFO - Trimming output
INFO - Successfully parsed input gwas in 2.0491462999925716 seconds
INFO - Started metaxcan process
INFO - Loading model from: ../data/PrediXcan/predictdb/en_Adipose_Subcutaneous.db
INFO - Loading covariance data from: ../data/PrediXcan/predictdb/en_Adipose_Subcutaneous.txt.gz
INFO - Processing loaded gwas
INFO - Started metaxcan association
INFO - 10 % of model's snps found so far in the gwas study
INFO - 20 % of model's snps found so far in the gwas study
INFO - 30 % of model's sn

<div class="alert alert-info">
    You can open one of the results files in the ``results`` directory to see the TWAS results for yourself.
</div>

In the [next notebook](1b-S-PrediXcan-interpretation.ipynb), we will interpret the TWAS results we generated using the R language.